# 4. 단어 추출 word extraction

## 4-a. 단어 추출을 하는 이유
- 커뮤니티 text는 신조어, 은어가 많다.
- 토큰화를 하기 위해선, 신조어와 은어를 추출해야 한다.

## 4-b. 단어 추출 계획
1. text로만 이루어진 txt 파일 만들기
2. soynlp 의 WordExtractor 를 이용하여 단어 추출
3. 단어와 수치 데이터를 csv 파일로 저장

## 4-c. 단어 추출 진행

### 4-c-1. 파일 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("3_spaced_text.csv", encoding="utf-8")
df.head(3)

,search_keyword,date_created,time_created,writer,is_reply,id,spaced_text
0,JYP,2023-05-16,17:52:40,ㅇㅇ(118.34),0,10100730087,야 개보지년들아 JYP 주가 봤냐.. 미쳣다 지금... 개잡주 오르는건 그러려니 하...
1,JYP,2023-05-16,17:52:48,ㅇㅇ(118.34),1,10100730087,ㅠㅠ
2,JYP,2023-05-16,18:04:04,ㅇㅇ(203.227),1,10100730087,ㅠㅠ


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891014 entries, 0 to 891013
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   search_keyword  891014 non-null  object
 1   date_created    891014 non-null  object
 2   time_created    891014 non-null  object
 3   writer          891014 non-null  object
 4   is_reply        891014 non-null  int64 
 5   id              891014 non-null  int64 
 6   spaced_text     891014 non-null  object
dtypes: int64(2), object(5)
memory usage: 47.6+ MB


### 4-c-2. corpus를 list 형태로 만든다

In [3]:
corpus = df['spaced_text'].tolist()

### 4-c-3. word_extractor 학습시키기

In [4]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(corpus) # list of str or like
words = word_extractor.extract()

training was done. used memory 2.502 Gbse memory 3.044 Gb
all cohesion probabilities was computed. # words = 27749
all branching entropies was computed # words = 1146192
all accessor variety was computed # words = 1146192


In [5]:
words['주식']

Scores(cohesion_forward=0.415731390877544, cohesion_backward=0.1350615196769043, left_branching_entropy=4.853872802910785, right_branching_entropy=4.579661565493153, left_accessor_variety=577, right_accessor_variety=561, leftside_frequency=39894, rightside_frequency=2876)

In [10]:
len(words)

14096

### 4-c-4. word extraction 결과 출력

In [11]:
# 결과 출력

import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy, word_score)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[1000:2000]:
    print('%s     (%d, %.3f, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy,
            word_score(score)
            )
         )

단어   (빈도수, cohesion, branching entropy, word_score)

마이크론도     (161, 0.275, 3.827, 12.639)
깨지면     (296, 0.294, 3.761, 12.633)
이제     (11713, 0.064, 5.282, 12.621)
성공하면     (262, 0.261, 3.879, 12.609)
개미가     (1075, 0.134, 4.547, 12.602)
냠냠     (260, 0.884, 2.656, 12.589)
백신     (1491, 0.280, 3.805, 12.584)
카카오게임즈는     (154, 0.366, 3.537, 12.580)
삼성은     (816, 0.116, 4.684, 12.580)
뇌피셜     (385, 0.501, 3.222, 12.568)
높은     (2523, 0.278, 3.811, 12.565)
턴어라운드     (155, 0.816, 2.734, 12.562)
점유율이     (226, 0.337, 3.616, 12.542)
과천은     (368, 0.188, 4.198, 12.541)
장기적으로는     (127, 0.360, 3.551, 12.525)
산업이     (354, 0.210, 4.089, 12.516)
받으면     (473, 0.193, 4.170, 12.514)
있을까?     (582, 0.184, 4.218, 12.503)
설마     (1135, 0.149, 4.432, 12.495)
나는     (4625, 0.060, 5.331, 12.479)
회장이     (477, 0.141, 4.479, 12.470)
IT     (1236, 0.334, 3.620, 12.464)
페이스북     (317, 0.466, 3.286, 12.460)
있음?     (1520, 0.128, 4.581, 12.458)
아닌가?     (594, 0.165, 4.324, 12.448)
잡아서     (261, 0.174, 4.269, 1

### 4-c-5. threshold 판단
- 눈으로 직접 살펴보며, 무의미한 결과를 거를 word_score의 threshold를 정한다
- 살펴본 결과. word_score >= 3.0 인 값만 유의미한 데이터로 간주한다
- 결과 dict : scored_words

In [100]:
threshold = 3.0
# threshold 를 기준으로 dictionary 생성
scored_words = {word: word_score(score) for word, score in words.items() if word_score(score) >= threshold}

print(scored_words)

{'~~': 72.85926999327718, '최대': 5.065257385040853, '봐도': 50.19493866157249, 'ㅈㄴ': 44.828555325501995, '늦게': 11.02216181776753, '등을': 4.496407810131742, '건설': 11.013556396496401, '콘솔': 8.16577504631527, 'vs': 70.36003386623821, '형이': 13.87026108373031, '되면': 15.741135506749922, '최근': 9.239925770477633, '어케': 3.0795625798769892, '금지': 4.305103490236238, '밴드': 5.508050184765097, '천억': 5.894281077558707, 'ㄱㄱ': 46.08873549234032, '니가': 56.75770317382826, '휴젤': 8.571650762659381, '겨우': 41.78707171467094, '물량': 9.042483320932062, '통수': 3.0563707821496005, '쇼크': 6.3731874424781605, '요즘': 83.67497867615428, '지수': 4.13234481914235, '막상': 9.852391072385416, '된거': 3.3805285060001506, '레알': 7.602321269313615, '기업': 12.182883889727899, '세금': 4.9486843420482405, '고점': 5.610908283358609, '절대': 19.955315986415496, '출시': 14.69461521305904, '잠시': 4.846210642890053, 'ㅆㅂ': 18.988416185636883, '주식': 40.52607699025103, 'QQ': 3.0837475246642008, '섹터': 31.534060436720786, '병신': 19.792527321743155, '게임': 58.224

In [120]:
# 리스트로 만든다
key_list = list(scored_words.keys())
len(key_list)

4378

### 4-c-6. 숫자 포함된 토큰 삭제

In [121]:
import re

def remove_items_with_numbers(list_a):
    # 정규 표현식을 사용하여 숫자 포함 여부 판단
    return [item for item in list_a if not re.search(r'\d', item)]

# 사용 예시
filtered_list_1 = remove_items_with_numbers(key_list)
len(filtered_list_1) 

4210

### 4-c-7. 조사/접미사 붙은 토큰들 삭제
- "이/가", "을/를", "은/는", "와/과", "도/만/조차", "에/에서/로", "의", "면", "까지/부터", "처럼", "다", "들"
- "." 붙은 것도 삭제
- 명사 토큰을 삭제해 버릴 지라도, mecab에서 처리해 주기 때문에 상관없다
- 기준을 엄격히 하여, 이상한 토큰으로 토큰화하지 않는 것이 중요하다. 단어의 원자성을 유지하자.

In [123]:
# 확인해보기
def find_endswith(input_list, endswith):
    result = [item for item in input_list if item.endswith(endswith)]
    return result
    
find_endswith(filtered_list_1, "만원")

['천만원', '십만원', '백만원']

In [140]:
def remove_items_with_suffixes(list_a, suffixes):
    # 접미사로 끝나는 원소들을 필터링하여 제거
    return [item for item in list_a if not item.endswith(tuple(suffixes))]

suffixes = ["이", "가", "을", "를", "은", "는", "와", "과", "의", "도", "만", "나", "면", "조차", "하고", "이나", "이냐", "이라",
            "듯", "다고", "게", "데", "지", "하냐", "면서", "?", "!", ".", ";", "-", "~", "/", "*", "라고", "요", "까", 
            "에", "에서", "로", "까지", "부터", "처럼", "다", "들", "들아", "야", "니까", "봐", "거", "건", "서", "음", "네", "원"]

filtered_list_2 = remove_items_with_suffixes(filtered_list_1, suffixes)
len(filtered_list_2)

1708

In [141]:
filtered_list_2[1000:]

['ㅎㅎㅎ',
 '싸이클',
 '문재인',
 '누구냐',
 '샤오미',
 '담아라',
 '대부분',
 'hmm',
 '태양광',
 'NHN',
 '이번엔',
 '헛소리',
 '가지고',
 '더이상',
 '순환매',
 '라이언',
 '언제쯤',
 '발표한',
 'LGD',
 '블로그',
 '선반영',
 '떨어짐',
 '삼스디',
 '드디어',
 '생산직',
 '전세계',
 '맥쿼리',
 '호가창',
 '메지온',
 '중공업',
 '먹거리',
 '머스크',
 '티슈진',
 '신라젠',
 'qld',
 '대규모',
 '완벽한',
 '시밀러',
 '네이버',
 '일자리',
 '김택진',
 '해주갤',
 'HBM',
 '대주주',
 '외제차',
 '업비트',
 '좆병신',
 '거래량',
 '쌉고수',
 '갭하락',
 '것이고',
 '성과금',
 '무지성',
 '하이브',
 '했잖아',
 '점유율',
 '때리고',
 '몸주남',
 '조금씩',
 '그동안',
 '수수료',
 '성장률',
 'txt',
 '솔직히',
 '알려줌',
 '고마워',
 '드라마',
 '당연히',
 '데이터',
 '차세대',
 '움직임',
 '타이밍',
 '만들고',
 '휴대폰',
 '나스닥',
 'skt',
 '성과급',
 'QQQ',
 '죽어라',
 '상당히',
 '소리임',
 '털리고',
 '최소한',
 '분위기',
 '리포트',
 '했더니',
 '닥버스',
 '신세계',
 '메모리',
 '전재산',
 '블록딜',
 '건설주',
 '아이돌',
 '여전히',
 '가즈아',
 'imf',
 '올라감',
 '강력한',
 '민주당',
 '찌라시',
 '퀘스트',
 '트위터',
 '데리고',
 '올리고',
 '아저씨',
 '몰빵한',
 '캐릭터',
 '대북주',
 '뇌피셜',
 '진정한',
 '박스권',
 '홀딩스',
 '확진자',
 '경쟁률',
 '평가좀',
 '메신저',
 '손절한',
 '순매수',
 '수익률',
 '월요일',
 '대기업',
 '거래소',
 '다팔고',
 'the',
 '힘내라',


## 4-d. 단어 리스트를 파일로 저장

### 4-e-1. 파일로 저장하기

In [142]:
# 파일로 저장
filename = "4_word_list.txt"
words = filtered_list_2
with open(filename, "w", encoding="utf-8") as file:
    for word in words:
        file.write(word + "\n")

print("파일 저장 완료.")

파일 저장 완료.


### 4-e-2. 불러와서 확인하기

In [143]:
# 파일로부터 문자열 리스트 불러오기
with open(filename, "r", encoding="utf-8") as file:
    word_list = [line.rstrip('\n') for line in file]

print("불러온 문자열 리스트:")
print(word_list)

불러온 문자열 리스트:
['최대', 'ㅈㄴ', '건설', '콘솔', 'vs', '최근', '어케', '밴드', '천억', 'ㄱㄱ', '휴젤', '겨우', '물량', '통수', '쇼크', '요즘', '지수', '막상', '레알', '기업', '세금', '고점', '절대', '출시', '잠시', 'ㅆㅂ', '주식', 'QQ', '섹터', '병신', '게임', 'it', '웹툰', '승인', 'YG', '횡보', '업계', '저런', '언제', '운영', '실적', '퍄퍄', '듣고', '스트', '이제', '각각', '친구', '굿굿', '사고', 'FT', '착한', '교육', '조정', '몇개', '잡고', '당시', 'NC', 'AI', '각종', '임상', '손절', '미장', '놓고', '증시', '세계', 'ㅇㅇ', '서울', '껄껄', '쌍봉', '시총', '획득', '쩜하', '<<', '코인', '머리', '매출', '괜히', '앨범', '셀트', '뭐냐', '랠리', '임마', '분할', '과연', '온갖', '줍줍', '떵락', '링크', '념글', '흑우', '행사', '격차', '길드', '금방', 'ㅁㅊ', '거품', '흐름', '본사', '위한', '형님', '합병', '환경', '음봉', '통해', '북한', '페북', '갱신', '손실', 'nc', '슈카', '활동', '서버', '엘화', '말고', '동안', '쉬운', '독일', '직접', '떨어', '삼전', '치고', '뜨고', '왜케', '사라', '평단', '책임', '택시', 'ㅋㅋ', '똑같', '방송', '얼마', '봐라', '금융', '투자', '방금', 'ㅠㅠ', '인생', '뚜잇', '큐브', '와라', '젭알', '효율', '붕괴', '품질', '지금', '반등', '능력', '경제', '모두', '옥션', '알고', '시장', '시발', '쭈욱', '조선', '교수', '평생', 'ㅌㅌ', '낙폭', '낚시', '암드', '관심', '광기', '한번', '델